In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 1. Xử lý dữ liệu thiếu (Missing Data)

In [3]:
import pandas as pd
import numpy as np

In [4]:
file_path = '/content/drive/MyDrive/Project_DA_TIMA/Data/Tima_CRM_RAW.csv'
df = pd.read_csv(file_path)
df.head()

,STT,SoTienDKVayBanDau,TienGiaiNgan,SoTienConLai,application_date,TS_CREDIT_SCORE_V2,Số điện thoại khách hàng,FromDate,ID,LoanID,...,InterestPaymentType,LongestOverdue,CreditInfo,Name,Address,CheckTime,Brieft,NumberOfLoans,HasBadDebt,HasLatePayment
0,2,10000000.0,NaN,0,2023-02-30,403,977966899,2016-06-24 00:00:00,40,16104,...,Lãi ngày,4,NaN,NaN,NaN,2019-03-17 11:22:15,notfound,-1,0,0
1,3,NaN,250000.0,0,2016-07-28,403,977966899,2016-07-28 00:00:00,226,20871,...,Lãi ngày,147,NaN,NaN,NaN,2019-03-17 11:22:15,notfound,-1,0,0
2,4,5000000.0,5000000.0,-50000,2016-06-29,531,397511119,2016-06-29 00:00:00,45,17049,...,Lãi ngày,3,"Khách hàng hiện không có quan hệ tại TCTD, khô...",DAO CONG QUYET,"P203-E9 PHUONG MAI,DONG DA,HA NOI",2019-03-17 08:29:24,0,0,0,0
3,5,10000000.0,10000000.0,0,2016-06-29,588,abc1234567,2016-06-29 00:00:00,48,17067,...,Lãi ngày,16,"Khách hàng hiện không có quan hệ tại TCTD, khô...",NGUYEN ANH VU,SO 21 NGACH 77/4 LE HONG PHONG-DIEN BIEN-BA DI...,2019-03-17 08:28:09,0,0,0,0
4,6,10000000.0,10000000.0,0,2016-08-05,675,966710180,2016-08-05 00:00:00,293,22390,...,Lãi ngày,8,"Khách hàng hiện không có quan hệ tại TCTD, khô...",ĐÀO NGỌC LINH,"-, TT CỤC KHÍ TƯỢNG THỦY VĂN, TRUNG LIỆT, Q.ĐỐ...",2019-03-17 14:14:22,0,0,0,0


In [5]:
# Kiểm tra thông tin cơ bản về dữ liệu và giá trị thiếu
missing_values = (df.isnull() | (df == '')).sum()
missing_values

,0
STT,0
SoTienDKVayBanDau,1
TienGiaiNgan,1
SoTienConLai,0
application_date,0
TS_CREDIT_SCORE_V2,0
Số điện thoại khách hàng,0
FromDate,0
ID,0
LoanID,0


In [6]:
# Drop blank values o mot so cot
df = df.dropna(subset=['Gender', 'CityName'])

In [7]:
unknown_missing_cols = ['Address', 'Name', 'FullNameFamily',
                        'RelativeFamilyName', 'WardName', 'Thời gian đã sống',
                        'Street', 'JobName', 'NameCompany', 'AddressCompany',
                        'CityCompany', 'DistrictNameCompany', 'DescriptionPositionJob']
df[unknown_missing_cols] = df[unknown_missing_cols].fillna('N/A')

In [8]:
from sklearn.impute import SimpleImputer
# Thay gia tri null o cot so bang trung binh
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
mean_imputer = SimpleImputer(strategy='mean')
df[numeric_cols] = mean_imputer.fit_transform(df[numeric_cols])

In [9]:
# Thay gia tri null o cot chu bang gia tri thuong xuyen xuat hien
str_cols = df.select_dtypes(include=['object']).columns.tolist()
freq_imputer = SimpleImputer(strategy='most_frequent')
df[str_cols] = freq_imputer.fit_transform(df[str_cols])

## 2. Xử lý giá trị ko hợp lệ

In [10]:
# Xóa cột trùng dữ liệu
col_del = [
    'SoTienDKVayBanDau',
    'TienGiaiNgan',
    'SoTienConLai',
]
cols_del_exist = [col for col in col_del if col in df.columns]
df.drop(columns=cols_del_exist, inplace = True)

In [11]:
initial_rows = len(df)
initial_cols = len(df.columns)
initial_rows, initial_cols

(2379, 46)

### Cột Salary

In [12]:
usd_rate = 26155
salary_map = {
    '1000$': 1000 * usd_rate,
    'Ten thousand': 10000 * usd_rate
}

df['Salary'] = df['Salary'].replace(salary_map)

### Cột LongestOverdue

In [13]:
invalid_remaining = df[df['Tiền gốc còn lại'] < 0]
invalid_longestOverdue = df[df['LongestOverdue'] < 0]
if not invalid_remaining.empty:
    print(f"{len(invalid_remaining)} hang co so tien con lai < 0 ")
    df = df[df['Tiền gốc còn lại'] >= 0]
    print(f"\nTìm thấy {len(invalid_longestOverdue)} hàng có 'LongestOverdue' < 0")
    print(invalid_longestOverdue[['STT', 'LongestOverdue']])
    df = df[df['LongestOverdue'] >=  0]
else:
    print(f"Khong co gia tri nao invalid")

Khong co gia tri nao invalid


### Định dạng Số điện thoại sai

In [14]:
import re

phone_pattern = re.compile(r'^(0?\d{9})$')

string_rules = {
    'Số điện thoại khách hàng': phone_pattern,
}

# Hàm xử lý regex
def validate_string_columns(df, rules, action='drop', verbose=True):
    df_out = df.copy()
    invalid_masks = {}

    for col, pat in rules.items():
        if col not in df_out.columns:
            if verbose:
                print(f" Cột '{col}' không tồn tại.")
            continue

        values = df_out[col].astype(str).str.strip()

        mask_invalid = ~values.str.match(pat)
        invalid_masks[col] = mask_invalid

        if action == 'nan':
            df_out.loc[mask_invalid, col] = np.nan
        elif action == 'drop':
            pass

        if verbose:
            print(f"Cột {col}: {mask_invalid.sum()} giá trị không hợp lệ.")

    if action == 'drop':
        combined_mask = pd.DataFrame(invalid_masks).any(axis=1)
        n_before = len(df_out)
        df_out = df_out.loc[~combined_mask].copy()
        n_after = len(df_out)
        if verbose:
            print(f"Đã loại {n_before - n_after} dòng do vi phạm quy tắc chuỗi.")

    return df_out

df = validate_string_columns(df, string_rules, action='drop')
df['Số điện thoại khách hàng'] = df['Số điện thoại khách hàng'].apply(lambda x: x if x.startswith('0') else '0' + x)

Cột Số điện thoại khách hàng: 1 giá trị không hợp lệ.
Đã loại 1 dòng do vi phạm quy tắc chuỗi.


## 3. Xử lý dữ liệu trùng lặp

In [15]:
# Xem dữ liệu SĐT trùng lặp
duplicates_phone = df[df.duplicated(subset=['CheckTime', 'CardNumber'], keep=False)]
duplicates_phone[['STT', 'FullName', 'CardNumber', 'CheckTime']]

,STT,FullName,CardNumber,CheckTime
0,2.0,Phạm Sơn Tùng,1.260311e+07,2019-03-17 11:22:15
1,3.0,Phạm Sơn Tùng,1.260311e+07,2019-03-17 11:22:15
4,6.0,Đào Ngọc Linh,1.182006e+09,2019-03-17 14:14:22
5,7.0,Đào Ngọc Linh,1.182006e+09,2019-03-17 14:14:22
7,9.0,Dương Minh Hà,1.091007e+09,2019-03-17 11:42:35
...,...,...,...,...
2038,2636.0,Vũ Thị Lan Anh,2.717300e+10,2019-03-17 12:22:36
2128,2749.0,Văn Thị Thủy Tiên,3.121827e+08,2019-03-17 09:49:30
2129,2750.0,Văn Thị Thủy Tiên,3.121827e+08,2019-03-17 09:49:30
2229,2887.0,Trần Văn Tâm,3.409600e+10,2019-03-17 14:23:10


In [16]:
# Trùng lặp hoàn toàn
df.drop_duplicates(keep='first', inplace=True)

In [17]:
# Trùng lặp vay
subset_cols = ['CardNumber', 'application_date']
print(f"Kiểm tra trùng lặp dựa trên các cột: {subset_cols}")

rows_before_subset_dedup = df.shape[0]
print(f"Số hàng trước khi xóa trùng lặp theo subset: {rows_before_subset_dedup}")

df_no_subset_duplicates = df.drop_duplicates(subset=subset_cols, keep='first')

rows_after_subset_dedup = df_no_subset_duplicates.shape[0]
print(f"Số hàng sau khi xóa trùng lặp theo subset: {rows_after_subset_dedup}")
print(f"==> Đã loại bỏ: {rows_before_subset_dedup - rows_after_subset_dedup} dòng.")

Kiểm tra trùng lặp dựa trên các cột: ['CardNumber', 'application_date']
Số hàng trước khi xóa trùng lặp theo subset: 2378
Số hàng sau khi xóa trùng lặp theo subset: 2378
==> Đã loại bỏ: 0 dòng.


## 4. Xử lý giá trị ngoại lai (IQR)

In [18]:
def detect_and_remove_outliers_iqr(data, column_name):
    column_data = data[column_name].dropna()
    Q1 = column_data.quantile(0.25)
    Q3 = column_data.quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    outliers = data[(data[column_name] < lower_bound) | (data[column_name] > upper_bound)]
    print(f"Đã xóa {len(outliers)} giá trị ngoại lệ.")

    return data[(data[column_name] >= lower_bound) & (data[column_name] <= upper_bound)]


In [19]:
df[['TS_CREDIT_SCORE_V2', 'LongestOverdue']].describe()

,TS_CREDIT_SCORE_V2,LongestOverdue
count,2378.000000,2378.000000
mean,589.649706,5.801514
std,97.027117,12.402592
min,300.000000,-23.000000
25%,534.250000,1.000000
50%,598.000000,3.000000
75%,660.000000,6.000000
max,826.000000,206.000000


In [20]:
df = detect_and_remove_outliers_iqr(df, 'TS_CREDIT_SCORE_V2')
df = detect_and_remove_outliers_iqr(df, 'LongestOverdue')

Đã xóa 50 giá trị ngoại lệ.
Đã xóa 194 giá trị ngoại lệ.


## 5. Chuẩn hóa dữ liệu (Data Standardization)

In [21]:
# Chuẩn hóa cột 'FullName' và 'Name' về dạng VIẾT HOA (UPPERCASE)
df['FullName'] = df['FullName'].str.upper()
df['Name'] = df['Name'].str.upper()
# Chuẩn hóa cột 'JobName' về dạng Title Case (Viết Hoa Chữ Cái Đầu)
df['JobName'] = df['JobName'].str.strip().str.title()

In [22]:
def truncate_columns(df, column_max_len):
    """
    column_max_len: dict với { 'column_name': max_length }
    """
    for col, max_len in column_max_len.items():
        if col in df.columns and df[col].dtype == object:
            df[col] = df[col].astype(str).str.slice(0, max_len)
    return df


df = truncate_columns(df, {'NameCompany': 50, 'DescriptionPositionJob': 100})


In [23]:
# Chuẩn hóa cột số bằng StandardScaler từ sklearn
from sklearn.preprocessing import StandardScaler

numeric_cols = ['TS_CREDIT_SCORE_V2', 'Salary', 'LongestOverdue']

# Chuyển đổi cột 'Salary' sang dạng số, các giá trị không hợp lệ sẽ thành NaN
df['Salary'] = pd.to_numeric(df['Salary'], errors='coerce')

# StandardScaler yêu cầu không có giá trị thiếu (NaN). Ta sẽ loại bỏ các hàng có NaN trong các cột này.
df_numeric = df.dropna(subset=numeric_cols).copy()
print(f"Số hàng còn lại để chuẩn hóa số: {len(df_numeric)}")

print("\nThống kê các cột số TRƯỚC khi chuẩn hóa:")
print(df_numeric[numeric_cols].describe())

# Áp dụng StandardScaler
# Khởi tạo scaler
scaler = StandardScaler()

# Fit và transform dữ liệu.
# .fit() -> học các tham số (mean, std) từ dữ liệu
# .transform() -> áp dụng công thức chuẩn hóa
# .fit_transform() -> làm cả hai bước trên
scaled_features = scaler.fit_transform(df_numeric[numeric_cols])

# Kết quả trả về là một mảng NumPy, chuyển nó thành DataFrame
df_scaled = pd.DataFrame(scaled_features,
                         columns=[f'{col}_scaled' for col in numeric_cols],
                         index=df_numeric.index) # Giữ lại index gốc để ghép nối

print("\nThống kê các cột số SAU khi chuẩn hóa:")
print(df_scaled.describe())
print("=> Lưu ý: Mean xấp xỉ 0 và Std xấp xỉ 1.")

# Ghép nối dữ liệu đã chuẩn hóa vào DataFrame gốc
df = pd.concat([df_numeric, df_scaled], axis=1)

print("\nCác cột đã chuẩn hóa:")
df_scaled.head()

Số hàng còn lại để chuẩn hóa số: 2134

Thống kê các cột số TRƯỚC khi chuẩn hóa:
       TS_CREDIT_SCORE_V2        Salary  LongestOverdue
count         2134.000000  2.134000e+03     2134.000000
mean           598.995314  9.613344e+06        3.251172
std             88.550361  8.480373e+06        3.254293
min            346.000000  0.000000e+00       -5.000000
25%            543.000000  6.500000e+06        1.000000
50%            607.000000  8.000000e+06        2.000000
75%            663.750000  1.100000e+07        5.000000
max            826.000000  2.615500e+08       13.000000

Thống kê các cột số SAU khi chuẩn hóa:
       TS_CREDIT_SCORE_V2_scaled  Salary_scaled  LongestOverdue_scaled
count               2.134000e+03   2.134000e+03           2.134000e+03
mean                1.331851e-17   7.325183e-17           6.659257e-18
std                 1.000234e+00   1.000234e+00           1.000234e+00
min                -2.857748e+00  -1.133865e+00          -2.536067e+00
25%                -6

,TS_CREDIT_SCORE_V2_scaled,Salary_scaled,LongestOverdue_scaled
0,-2.213896,29.715167,0.230159
2,-0.768052,1.951038,-0.077200
4,0.858523,0.281499,1.459592
5,0.858523,0.281499,-0.691917
6,-0.191973,-0.190289,1.152234


## 6. Chuyển đổi kiểu dữ liệu (Data Type Conversion)

In [24]:
# Chuyển đổi cột 'application_date'
df['application_date'] = pd.to_datetime(df['application_date'], errors='coerce')
df['Birthday'] = df['Birthday'].astype(str).str[:10]
df['Birthday'] = pd.to_datetime(df['Birthday'],  errors='coerce')
df['FromDate'] = pd.to_datetime(df['FromDate'], errors='coerce')
df['ToDate']   = pd.to_datetime(df['ToDate'], errors='coerce')

print(df[['application_date', 'Birthday', 'FromDate', 'ToDate']].dtypes)

application_date    datetime64[ns]
Birthday            datetime64[ns]
FromDate            datetime64[ns]
ToDate              datetime64[ns]
dtype: object


/tmp/ipython-input-1169598855.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['application_date'] = pd.to_datetime(df['application_date'], errors='coerce')


In [25]:
df[df['application_date'].isna()]

,STT,application_date,TS_CREDIT_SCORE_V2,Số điện thoại khách hàng,FromDate,ID,LoanID,Số tiền đăng ký vay ban đầu,Tiền giải ngân,Tiền gốc còn lại,...,Name,Address,CheckTime,Brieft,NumberOfLoans,HasBadDebt,HasLatePayment,TS_CREDIT_SCORE_V2_scaled,Salary_scaled,LongestOverdue_scaled
0,2.0,NaT,403.0,0977966899,2016-06-24,40.0,16104.0,10000000.0,10000000.0,10000000.0,...,N/A,N/A,2019-03-17 11:22:15,notfound,-1.0,0.0,0.0,-2.213896,29.715167,0.230159


In [26]:
df = df.dropna(subset=['application_date'])

In [27]:
df.dtypes

,0
STT,float64
application_date,datetime64[ns]
TS_CREDIT_SCORE_V2,float64
Số điện thoại khách hàng,object
FromDate,datetime64[ns]
ID,float64
LoanID,float64
Số tiền đăng ký vay ban đầu,float64
Tiền giải ngân,float64
Tiền gốc còn lại,float64


## 7. Tạo biến mới (Feature Engineering)

In [28]:
# Tạo cột 'CustomerAge' (Tuổi của khách hàng tại thời điểm nộp đơn)
print("\nTạo cột 'CustomerAge' từ 'Birthday' và 'application_date'")
# Tính tuổi bằng cách lấy ngày nộp đơn trừ ngày sinh và chia cho 365.25 (tính cả năm nhuận)
# .astype('int') để làm tròn tuổi thành số nguyên
# Xử lý trường hợp ngày sinh hoặc ngày nộp đơn không hợp lệ (NaT)
df['CustomerAge'] = ((df['application_date'] - df['Birthday']).dt.days / 365.25).fillna(-1).astype('int')
# Thay thế các giá trị tuổi vô lý (âm) bằng NaN để dễ dàng xử lý sau này
df['CustomerAge'] = df['CustomerAge'].apply(lambda x: x if x > 0 else np.nan)
print(df[['STT', 'FullName', 'Birthday', 'application_date', 'CustomerAge']].head())


Tạo cột 'CustomerAge' từ 'Birthday' và 'application_date'
   STT        FullName   Birthday application_date  CustomerAge
2  4.0  ĐÀO CÔNG QUYẾT 2025-01-01       2016-06-29          NaN
4  6.0   ĐÀO NGỌC LINH 1982-01-31       2016-08-05         34.0
5  7.0   ĐÀO NGỌC LINH 1982-01-31       2016-06-29         34.0
6  8.0   ĐỖ MINH TRANG 1988-01-01       2016-07-07         28.0
7  9.0   DƯƠNG MINH HÀ 1991-03-31       2018-05-31         27.0


In [29]:
# Tạo cột 'LoanDuration' (Thời hạn vay tính bằng ngày)
print("Tạo cột 'LoanDuration' từ 'ToDate' và 'FromDate'")
# Phép trừ giữa hai cột datetime sẽ trả về một đối tượng Timedelta
df['LoanDuration'] = (df['ToDate'] - df['FromDate']).dt.days
print(df[['STT', 'FromDate', 'ToDate', 'LoanDuration']].head())

Tạo cột 'LoanDuration' từ 'ToDate' và 'FromDate'
   STT   FromDate     ToDate  LoanDuration
2  4.0 2016-06-29 2016-12-10           164
4  6.0 2016-08-05 2016-12-02           119
5  7.0 2016-06-29 2016-07-28            29
6  8.0 2016-07-07 2017-03-03           239
7  9.0 2018-05-31 2018-08-28            89



## Đổi tên 1 số cột & Xuất File

In [30]:
df = df.rename(columns={
    "Số tiền đăng ký vay ban đầu": "SoTienDKVayBanDau",
    "Tiền giải ngân": "TienGiaiNgan",
    "Tiền gốc còn lại": "TienGocConLai",
    "Trạng thái": "TrangThai",
    "Số điện thoại khách hàng": "SoDienThoai",
    "Thời gian đã sống": "ThoiGianDaSong",
    "Hình thức cư trú" : "HinhThucCuTru"
})

In [31]:
df.dropna(inplace=True)
df.to_csv('/content/drive/MyDrive/Project_DA_TIMA/Data/Tima_CRM_Handled_Python.csv', index=False, encoding='utf-8-sig')

In [32]:
df.shape

(2131, 51)